In [1]:
import pandas as pd
import numpy as np

from sklearn.base import clone
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold, cross_val_score
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV

# Scikit-learn Metrics
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, accuracy_score, pairwise_distances, confusion_matrix, make_scorer, roc_curve, f1_score
import sklearn.metrics as skm

# Scikit-learn Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Combined from both lists
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier

from imblearn.combine import SMOTEENN

In [2]:
data = pd.read_csv("data/processed/fraud.csv")
data.shape

(15420, 45)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(
    ['FraudFound_P'], axis=1), data['FraudFound_P'], test_size=0.2, random_state=42, stratify=data['FraudFound_P'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


In [4]:
import lightgbm as lgb

# LigthGBM
lightgbm_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, verbose=-1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rfecv = RFECV(estimator=lightgbm_model, step=1, cv=skf, scoring='f1', n_jobs=-1)
rfecv.fit(X_train, y_train)

,estimator,"LGBMClassifie...2, verbose=-1)"
,step,1
,min_features_to_select,1
,cv,StratifiedKFo... shuffle=True)
,scoring,'f1'
,verbose,0
,n_jobs,-1
,importance_getter,'auto'
,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1


In [5]:
optimal_num_features =rfecv.n_features_
print(f"Optimal number of features: {optimal_num_features}")

Optimal number of features: 27


In [6]:
X_selected = rfecv.transform(X_train)

In [7]:
selected_feature_indices = np.where(rfecv.support_)[0]
selected_feature_names = X_train.columns[selected_feature_indices]
X_rfecv = pd.DataFrame(X_selected, columns=selected_feature_names)

In [8]:
X_rfecv.head()

,Month,WeekOfMonth,DayOfWeek,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,...,PastNumberOfClaims,AgeOfVehicle,AgentType,NumberOfSuppliments,AddressChange_Claim,BasePolicy,AgeVehicleRatio,NoPoliceReport,Make_label_encoded,Make_frequency_encoded
0,12.0,3.0,5.0,1.0,2.0,12.0,4.0,1.0,0.0,26.0,...,3.0,6.0,0.0,0.0,0.0,2.0,4.714286,1.0,13.0,0.248833
1,8.0,2.0,2.0,1.0,3.0,8.0,2.0,1.0,0.0,61.0,...,3.0,8.0,0.0,6.0,0.0,2.0,6.444444,1.0,6.0,0.181647
2,11.0,4.0,6.0,1.0,1.0,11.0,4.0,1.0,0.0,68.0,...,3.0,8.0,0.0,0.0,0.0,1.0,7.777778,1.0,13.0,0.248833
3,6.0,4.0,4.0,1.0,5.0,6.0,4.0,1.0,0.0,52.0,...,5.0,7.0,0.0,4.0,0.0,1.0,5.750000,1.0,6.0,0.181647
4,4.0,2.0,6.0,1.0,3.0,4.0,2.0,0.0,0.0,42.0,...,3.0,6.0,0.0,6.0,0.0,1.0,5.428571,1.0,17.0,0.202399


In [9]:
model_SVC = SVC(cache_size = 500)

svc_model = Pipeline([
    ('smoteenn', SMOTEENN(random_state=42)),
    ('pca', PCA(n_components=0.95, random_state=42)),
    ('svc', model_SVC)
])

cv_scores = cross_val_score(svc_model , X_rfecv, y_train, cv=skf, scoring='f1', n_jobs=-1)
print("Baseline CV F1-mean:", cv_scores.mean())
print("Baseline CV F1-std:", cv_scores.std())

Baseline CV F1-mean: 0.11283228769752549
Baseline CV F1-std: 2.6430288932761298e-05


In [10]:
!pip install optuna


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [12]:
def objective(trial):

  param_grid = [{'svc__kernel': ["linear"], 'svc__gamma': [0.1, 1, 10, 100], 'svc__C': [0.1, 1, 10, 100, 1000], 'svc__degree': [0, 1, 2, 3, 4, 5, 6],
                'svc__random_state': [24]}]

  params = {
      'gamma': trial.suggest_float('gamma', 0, 1),
      'C': trial.suggest_float('C', 10, 30),
      'degree': trial.suggest_int('degree', 50, 75),
      'cache_size' : 500,
      'random_state': 42
  }


  model = SVC(**params)
  pipeline = Pipeline([
      ('sampling', SMOTEENN(random_state=42)),
      ('pca', PCA(n_components=0.95, random_state=42)),
      ('classification', model)
  ])

  stratified_kfold = StratifiedKFold(n_splits=3,
                                  shuffle=True,
                                  random_state=42)
  f1_scorer = make_scorer(f1_score, average="macro")
  score = cross_val_score(pipeline, X_rfecv, y_train,
                          scoring=f1_scorer, cv=stratified_kfold, n_jobs=-1)
  auc_mean = np.mean(score)
  return auc_mean

study_SVC = optuna.create_study(direction='maximize', sampler=TPESampler())
study_SVC.optimize(objective, n_trials=100, show_progress_bar=True, n_jobs=-1)

[I 2025-11-19 16:00:55,229] A new study created in memory with name: no-name-34f2fcfa-7852-4ba7-a55a-da2ad00d97a4


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-11-19 16:01:14,022] Trial 1 finished with value: 0.27579724089218705 and parameters: {'gamma': 0.5973051658117658, 'C': 28.062745920632636, 'degree': 65}. Best is trial 1 with value: 0.27579724089218705.


[I 2025-11-19 16:01:14,694] Trial 2 finished with value: 0.24951946616704848 and parameters: {'gamma': 0.8641937724580616, 'C': 12.961465543593567, 'degree': 74}. Best is trial 1 with value: 0.27579724089218705.


[I 2025-11-19 16:01:14,964] Trial 3 finished with value: 0.32683711435736634 and parameters: {'gamma': 0.26839852204856585, 'C': 12.62509974728645, 'degree': 73}. Best is trial 3 with value: 0.32683711435736634.


...

[I 2025-11-19 16:05:58,163] Trial 94 finished with value: 0.366904377206676 and parameters: {'gamma': 0.014723298265613898, 'C': 26.95286962485547, 'degree': 64}. Best is trial 22 with value: 0.3671539729378641.


[I 2025-11-19 16:06:02,396] Trial 95 finished with value: 0.3671833519522168 and parameters: {'gamma': 0.01466204482635828, 'C': 25.565742059378948, 'degree': 70}. Best is trial 95 with value: 0.3671833519522168.


[I 2025-11-19 16:06:04,013] Trial 97 finished with value: 0.3614585164761022 and parameters: {'gamma': 0.0010588579498223942, 'C': 18.29548947955247, 'degree': 50}. Best is trial 95 with value: 0.3671833519522168.
[I 2025-11-19 16:06:04,145] Trial 96 finished with value: 0.35296056722553165 and parameters: {'gamma': 0.12440105616457522, 'C': 18.250150544071325, 'degree': 64}. Best is trial 95 with value: 0.3671833519522168.


[I 2025-11-19 16:06:08,576] Trial 98 finished with value: 0.35273791804644733 and parameters: {'gamma': 0.12738112388788533, 'C': 15.593024327130308, 'degree': 50}. Best is trial 95 with value: 0.3671833519522168.


[I 2025-11-19 16:06:10,704] Trial 99 finished with value: 0.36609853723112423 and parameters: {'gamma': 0.005696178248365969, 'C': 28.674845607339485, 'degree': 50}. Best is trial 95 with value: 0.3671833519522168.


In [13]:
sv = study_SVC.best_params
sv['probability'] = True
print(sv)

{'gamma': 0.01466204482635828, 'C': 25.565742059378948, 'degree': 70, 'probability': True}


In [14]:
best = SVC(**sv)
best.fit(X_train, y_train)

y_pred_svc = best.predict(X_val)
y_proba_svc = best.predict_proba(X_test)[:,1]

print("best params:", study_SVC.best_params)
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_proba_svc):.4f}")
print(f"PR-AUC Score: {average_precision_score(y_test, y_proba_svc):.4f}")
print("Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_val, y_pred_svc)))
print("Classification Report:")
pd.DataFrame(classification_report(y_val, y_pred_svc, output_dict=True)).transpose()

best params: {'gamma': 0.01466204482635828, 'C': 25.565742059378948, 'degree': 70}
ROC-AUC Score: 0.5923
PR-AUC Score: 0.1054
Confusion Matrix:
      0  1
0  2318  2
1   148  0
Classification Report:


,precision,recall,f1-score,support
0,0.939984,0.999138,0.968659,2320.000000
1,0.000000,0.000000,0.000000,148.000000
accuracy,0.939222,0.939222,0.939222,0.939222
macro avg,0.469992,0.499569,0.484329,2468.000000
weighted avg,0.883615,0.939222,0.910570,2468.000000


In [15]:
print("hello")

hello
